In [1]:
# Package installations to work on WIRE

! pip install gensim
! pip install nltk
! pip install sentence_transformers
! pip install spacy
! python3 -m spacy download en_core_web_sm
! pip install bertopic
! pip install langchain


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
2023-08-25 17:21:59.422111: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 17:22:00.125080: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.8 MB/s eta

In [1]:
# imports

import csv
import json
import os
import re
import time
import unicodedata
import sys
import string
import pickle
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
ps = nltk.porter.PorterStemmer()

from sentence_transformers import SentenceTransformer
import spacy

from itertools import combinations
from collections import Counter
from scipy import stats

from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance, LangChain, TextGeneration
from hdbscan import HDBSCAN
import umap

import openai
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from langchain import OpenAI, PromptTemplate, LLMChain, HuggingFacePipeline, FewShotPromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.docstore.document import Document
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import SpectralEmbedding
from contextlib import contextmanager
from copy import deepcopy
from random import *
from pprint import pprint
from tqdm import tqdm
nltk.download(["names", "stopwords", "state_union", "twitter_samples", "movie_reviews", "averaged_perceptron_tagger", "vader_lexicon", "punkt", "wordnet"])

C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-pac

True

## Prepare LLM Model
- Create the Prompt
- Create an LLM (more than one option)
    - using a local LLM via LangChain's interface to HuggingFace
    - using GPT-4 as a chat completion through Azure

In [3]:
# Create a generic prompt template

template = '''
The following sentences are from a cable news transcript. Please classify whether the statements are POSITIVE, NEUTRAL, or NEGATIVE towards "{subject}". Only return the stance.

sentences:
{statements}
'''

template2 = '''
The following sentences are from a cable news transcript. Is "{subject}" a main subject in the sentences? For each sentence, please respond only YES or NO.

sentences:
{statements}
'''

template3 = '''
The following sentences are from a cable news transcript. Is "{subject}" the main subject in the sentences? If not, respond NO. If it is a main subject, please return whether the statements are POSITIVE, NEUTRAL, or NEGATIVE towards "{subject}".
For each sentence, respond with either NO, POSITIVE, NEUTRAL, or NEGATIVE.

sentences:
{statements}
'''

template4 = '''
The following sentences are from a cable news transcript. Is "{subject}" the main subject in the sentences? If not, respond NO. If it is a main subject, please return whether the statements are POSITIVE, NEUTRAL, or NEGATIVE towards "{subject}".'

For each sentence, respond with either NO, POSITIVE, NEUTRAL, or NEGATIVE. Below are examples with "Donald Trump" and their outputs:

Donald Trump is the greatest president
output: POSITIVE

People that attack Trump are deranged
output: POSITIVE

The investigations into Trump are baseless
output: POSITIVE

I think Trump is exactly where he should be: in jail
output: NEGATIVE

Trump's Mar-a-lago resort is in Florida
output: NEUTRAL

They think that an electoral system that allowed Trump to win has to be scrapped altogether
output: NO

sentences:
{statements}
'''

stance_prompt = PromptTemplate(
    input_variables=["subject","statements"],
    template=template
)

relevance_prompt = PromptTemplate(
    input_variables=["subject","statements"],
    template=template2
)

double_prompt = PromptTemplate(
    input_variables=["subject","statements"],
    template=template4
)

LangChain's interface to HuggingFace

In [4]:
# Specify the LLM to use, Langchain verison

llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-gpt4-xl", task = 'text2text-generation', device=0,
                                      model_kwargs={"max_length":500, "no_repeat_ngram_size":2})

KeyboardInterrupt: 

GPT-4 as a chat completion through Azure

In [4]:
with open("aci_openai_credentials.txt") as f:
    open_api_base, open_api_key = f.read().split("\n")

openai.api_type = "azure"
openai.api_base = open_api_base
openai.api_version = "2023-03-15-preview"
openai.api_key = open_api_key

Worked examples

In [5]:
s = ["Those fires - Trump Didn't do that, ANTIFA did!",
     "Donald Trump, with his business-oriented approach and distinct style, brought a fresh perspective to the political landscape, which many found refreshing and inspiring.",
     "Many admired Donald Trump's determination in policy implementation, viewing his bold decisions as a testament to strong leadership.",
     "Donald Trump's presidency was marked by his unapologetic demeanor and unique communication style, which some found off-putting and divisive.",
     "There were concerns about Donald Trump's approach to international relations, with critics pointing to an alleged lack of diplomacy and tact.",
     "Supporters of Donald Trump appreciated his focus on American jobs and his efforts to stimulate the economy, seeing him as a president who truly prioritized the American workforce.",
     "Critics of Donald Trump often highlighted his handling of environmental issues, expressing disappointment over his decision to withdraw from the Paris Agreement.",
     "French meringue trumps swiss meringue"
    ]

In [10]:
response = openai.ChatCompletion.create(
                                        engine="ACI_GPT-4",
                                        messages = [{"role":"user","content":double_prompt.format(subject = "Donald Trump", statements= s)}],
                                        temperature=0.90,
                                        max_tokens=800,
                                        top_p=0.95,
                                        frequency_penalty=0,
                                        presence_penalty=0,
                                        stop=None)

In [11]:
relevance = response.choices[0].message['content']
print(relevance)

["POSITIVE", "POSITIVE", "POSITIVE", "NEGATIVE", "NEGATIVE", "POSITIVE", "NEGATIVE", "NO"]


## Organizing Transcript-Topics

In [15]:
#set up llm df
llm_df = pd.DataFrame(columns = ['subject','sentence','output'])
def transcript_cleaning(month,gl,txt):
    counter = 0
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], [], []]

    #read fox information, this is where ****file names**** and directories should be edited
    gl = pd.read_csv(gl, encoding="windows-1252")
    fox = open(txt, "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt): 
            print(tr_tp_st)    
            counter += 1
            print(f"We're starting transcript #{counter}")
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            sentences = lines.explode('Sentence')
            sentences = sentences.dropna(subset = ['Sentence'])

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                #excluding unrelevant entities
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            print("number of most common words for classifying stance on: {}".format(len(words)))
            print(words)
            words = words[0:5]
            for w in words:
                #try:
                    #create df of all sentences including word
                    w1 = w[0]
                    wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]
                    wdf = wdf.reset_index()
                    if len(wdf) > 2: #need at least three sentences to be added as an entity
                        w_list.append(w1)
                        sentence_list = list(wdf['Sentence'])
                        print("number of all sentences feauring word '{}': {}".format(w1, len(sentence_list)))
                        sent_list = []
                        for s in [sentence_list[idx:idx+100] for idx in range(0, len(sentence_list), 100)]:
                            try:
                                #print(len(s))#r['Sentence'])
                                """s2 = ''
                                for sentence in s:
                                    s2 = s2 + '"' + sentence + '"' + '\n'"""
                                print("number of sentences: {}".format(len(s)))
                                if len(s) > 20:
                                    shuffle(s)
                                    s = s[0:20]
                                print(w1)
                                response = openai.ChatCompletion.create(
                                        engine="ACI_GPT-4",
                                        messages = [{"role":"user","content":double_prompt.format(subject = w1, statements= s)}],
                                        temperature=0.99,
                                        max_tokens=800,
                                        top_p=0.95,
                                        frequency_penalty=0,
                                        presence_penalty=0,
                                        stop=None)
                                output = response.choices[0].message['content']
                                output_list = output.split('\n')
                                if len(output_list) == 1:
                                    output_list = output.split(',')
                                print(f"There were {len(output_list)} returned lines")
                                #print(output)
                                # Filter the s list based on the relevance list (if first thre characters are 'yes')
                                filtered_s = [r for r in output_list if 'NO' not in r]
                                print("number of sentences after filtering: {}".format(len(filtered_s)))
                                #mapping llm output to int and storing
                                for a, b in zip(s, output_list):
                                    print(a)
                                    print(b)
                                    print('\n')
                                def stance_value(stance):  
                                        if 'NEGATIVE' in stance:  
                                            return -1  
                                        elif 'POSITIVE' in stance:  
                                            return 1  
                                        elif 'NEUTRAL' in stance:  
                                            return 0  
                                        else:  
                                            return "No sentiment recongized in the stance: "  
                                for stance in filtered_s:
                                    stance1 = stance_value(stance)
                                    try:
                                        stance1 = int(stance1)
                                        sent_list.append(stance1)
                                    except:
                                        print(stance1 + stance)
                                    #print(stance)
                                    #print(stance1)
                                    llm_df.loc[len(llm_df)] = [w1,s,stance]
                                time.sleep(1.5)
                            except Exception as error:
                                print("Error:", error)
                                time.sleep(5)
                        #print(sent_list)
                        #print(np.mean(sent_list))
                        word_sentiment_list.append(sent_list)
                #except Exception as error:
                #    print("Error:", error)

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    try:
                        tr_tp_st[0].append(l)
                        tr_tp_st[1].append(w_list[w])
                        tr_tp_st[2].append(word_sentiment_list[w])#np.mean(word_sentiment_list[w]))
                        tr_tp_st[3].append(lines["Program"][0])
                    except:
                        print("SOMETHING HAPPENED, WORD ADDING FAILED")

    return tr_tp_st

In [16]:
# Save out processed and topic modeled data
months = ['February']#['January','February','March','April','May','June','July','August','September','October','November','December']
for m in months:
    tr_tp_st = transcript_cleaning(m,"../Data/foxGuestList.csv","../Data/fox_text.txt")
    #tr_tp_st = transcript_cleaning(m,"../Data/MSNBCGuestList.csv","../Data/MSNBC_text.txt")
    tdf = pd.DataFrame(zip(tr_tp_st[0], tr_tp_st[1], tr_tp_st[2], tr_tp_st[3],tr_tp_st[4]), 
                       columns=["Transcript", "Topic", "Sentiment", "Program","Date"],)
    tdf.to_csv('processed_transcripts_llm_' + m + '2020.csv')

[[], [], [], [], []]
We're starting transcript #1
number of most common words for classifying stance on: 220
[('Andy Biggs', 21), ('Trump', 17), ('Russian', 17), ('Sean', 16), ('Bernie Sanders', 15), ('Democrats', 11), ('Bernie', 11), ('Julian Assange', 10), ('Stone', 10), ('Russia', 10), ('Hillary Clinton', 9), ('Roger Stone', 9), ('Democratic', 9), ('South Carolina', 9), ('Castro', 8), ('FISA', 8), ('Bolshevik Bernie', 6), ('American', 6), ('Joe Biden', 6), ('Bloomberg', 5), ('China', 5), ('America', 5), ('Congress', 5), ('Obama', 5), ('the Espionage Act', 5), ('Fidel Castro', 5), ('Cuba', 4), ('Joe', 4), ('CNN', 4), ('Marxist', 4), ('U.S.', 4), ('Hillary', 4), ('Dan Bongino', 4), ('Barr', 4), ('Sotomayor', 4), ('Ginsburg', 4), ('the Democratic Party', 3), ('Florida', 3), ('Nordic', 3), ('Assange', 3), ('Christopher Steele', 3), ('Geraldo', 3), ('FBI', 3), ('Biden', 3), ('FOX News', 3), ('Michael Bloomberg', 3), ('Bill Barr', 3), ('Karl Rove', 3), ('CDC', 3), ('Schumer', 3), ('Justin

KeyboardInterrupt: 

In [1]:
llm_df.to_csv("sentence_labels_October.csv")

NameError: name 'llm_df' is not defined

## Alternative transcrip sorting (using nouns and lines)

In [42]:
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    #nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            #lines["VADER"] = lines["Line"].apply(
            #    lambda x: list(sid.polarity_scores(x).values())[3]
            #)

            #getting most used nouns
            """word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1"""
            
            #getting most used known entities
            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:15]

           #print(len(words))
            for w in words:
                #try:
                    w1 = w[0]
                    #print(w1)
                    wdf = lines[lines['Line'].astype(str).str.contains(str(w1))]
                    wdf = wdf.reset_index()
                    print(len(wdf))
                    w_list.append(w1)
                    sent_list = []
                    for i, r in wdf.iterrows():
                        if i < 5:
                            #r = wdf[i]
                            print(r['Line'])
                            print(w1)
                            stance = llm(stance_prompt.format(subject = w1, statement=r['Line']))
                            d = {'AGAINST': -1, 'NEUTRAL': 0, 'FOR': 1}
                            stance1 = d[stance]
                            sent_list.append(stance1)
                            print(stance)
                            print(stance1)
                    word_sentiment_list.append(sent_list)
                #except:
                #    pass

                
            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])


    return tr_tp_st